# Step 0 
1. Create a cohort browser with the columns eid and p41270 (all ICD diagnoses) 
2. Use table exporter to export this cohort to a CSV  

# Step 1 - pulling individual codes
Process the update the list of all ICD diagnoses into one row per code (multiple rows for one person) 
</br>
</br>From Gracelyn_ICD10_Parsing

In [ ]:
import pandas as pd
import numpy as np
from time import time 
from multiprocessing import Pool

In [ ]:
# this file is the one you made during step 0 
!dx download 'data/ICD10_dates/icd10_codes_20231002_participant.csv'

In [ ]:
#Load file created by cohort browser
df_orig = pd.read_csv("icd10_codes_20231002_participant.csv")
df_orig

In [ ]:
#First split by "|" 
#thanks StackOverflow! ^_^ https://stackoverflow.com/questions/50731229/split-cell-into-multiple-rows-in-pandas-dataframe
tic = time()
df_new = (df_orig.set_index(['eid'])
   .apply(lambda x: x.str.split('|').explode())
   .reset_index())  
toc = time() 
print(f'Time elapsed: {toc-tic} seconds')
df_new

In [ ]:
#Now split by space
tic = time()
df_new[['p41270', 'B']] = df_new['p41270'].str.split(' ', 1, expand=True)
toc = time()
print(f'Time elapsed: {toc-tic} seconds')
df_new

In [ ]:
#Rename columns
df_new = df_new.rename(columns = {'eid':'ID', 'p41270':'ICD10', 'B':'description'})
df_new

In [ ]:
df_new['ICD10-group'] = df_new['ICD10']
df_new[['ICD10-group', 'B']] = df_new['ICD10-group'].str.split('.', 1, expand=True)
df_new

In [ ]:
#Keep the groups we are interested in 
group_list = ['G47', 'F51']
df_new = df_new[df_new['ICD10-group'].isin(group_list)]
df_new

In [ ]:
df_new['ICD10-group'].value_counts()

In [ ]:
for group in group_list:
    df = df_new[df_new['ICD10-group'] == group]
    df = df[['ID', 'ICD10']]
    print(df.ICD10.value_counts())
    df.to_csv(f'{group}_individual_codes.csv', header = True, index = False)

In [ ]:
for group in group_list:
    print(f'!dx upload {group}_individual_codes.csv --path data/ICD10_dates/{group}/{group}_individual_codes.csv')

In [ ]:
!dx upload G47_individual_codes.csv --path data/ICD10_dates/G47/G47_individual_codes.csv
!dx upload F51_individual_codes.csv --path data/ICD10_dates/F51/F51_individual_codes.csv

# Step 2 - pulling group ICD10 codes, ie. G47

## Need Spark Notebook 

In [ ]:
#setup - packages & env
import pyspark
import dxdata
import dxpy
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [ ]:
#setup - grabbing dataset
dispensed_database_name = dxpy.find_one_data_object(classname="database", name="app*", folder="/", name_mode="glob", describe=True)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(typename="Dataset", name="app*.dataset", folder="/", name_mode="glob")["id"]

In [ ]:
# the participant dataset is the one we ultimately want to work with 
dataset = dxdata.load_dataset(id=dispensed_dataset_id)
participant = dataset["participant"]

In [ ]:
# Pull down the fields we need 
#Here I want G47 and F51 
field_names = ["eid", 'p131060', 'p130920']
df = participant.retrieve_fields(names=field_names, coding_values="replace", engine=dxdata.connect())

In [ ]:
# Send to Pandas 
df = df.toPandas()

In [ ]:
# Human readable columns please
df = df.rename(columns={'eid':'ID',
                        'p131060':'G47_DATE',
                        'p130920':'F51_DATE'})

df

In [ ]:
group_list

In [ ]:
for group in group_list:
    test = df[~df[f'{group}_DATE'].isna()]
    test = test[['ID', group + '_DATE']]
    print(group, len(test))
    test.to_csv(f'{group}_with_date.csv', header = True, index = None)

In [ ]:
test

In [ ]:
for group in group_list:
    print(f'!dx upload {group}_with_date.csv --path data/ICD10_dates/{group}/{group}_with_date.csv')

In [ ]:
!dx upload G47_with_date.csv --path data/ICD10_dates/G47/G47_with_date.csv
!dx upload F51_with_date.csv --path data/ICD10_dates/F51/F51_with_date.csv

# Step 3 -- creating individual ICD10 files, ie. G47.3

In [ ]:
icd = 'G47'
group = pd.read_csv(f'{icd}_with_date.csv')
group = group[['ID', f'{icd}_DATE']]
individual = pd.read_csv(f'{icd}_individual_codes.csv')
print(icd)
print('group', len(group))
print('individual', len(individual))

In [ ]:
print(group.head())

In [ ]:
print(individual.head())

In [ ]:
#Find any IDs with more than 1 code
individual.ID.value_counts()

In [ ]:
df = group.merge(individual, left_on = 'ID', right_on = 'ID', how = 'left')
df

In [ ]:
test = df[~df['ICD10'].isna()]
codes_list = list(set(list(test['ICD10'])))
codes_list

In [ ]:
codes_with_data = []
for code in codes_list:
    df2 = df[df['ICD10'] == code]
    df2[f'{code}_DATE'] = df2[f'{icd}_DATE']
    df2 = df2[['ID', code + '_DATE']]
    print(code, len(df2))
    
    #Only create files for codes with at least 5 samples
    
    if len(df2) > 4:
        codes_with_data.append(code)
        df2.to_csv(f'{code}_with_date.csv', header = True, index = None)
    else:
        pass
    

In [ ]:
for code in codes_with_data:
    print(f'!dx upload {code}_with_date.csv --path data/ICD10_dates/{icd}/{code}_with_date.csv')

In [ ]:
!dx upload G47.2_with_date.csv --path data/ICD10_dates/G47/G47.2_with_date.csv
!dx upload G47.3_with_date.csv --path data/ICD10_dates/G47/G47.3_with_date.csv
!dx upload G47.0_with_date.csv --path data/ICD10_dates/G47/G47.0_with_date.csv
!dx upload G47.4_with_date.csv --path data/ICD10_dates/G47/G47.4_with_date.csv
!dx upload G47.1_with_date.csv --path data/ICD10_dates/G47/G47.1_with_date.csv
!dx upload G47.8_with_date.csv --path data/ICD10_dates/G47/G47.8_with_date.csv
!dx upload G47.9_with_date.csv --path data/ICD10_dates/G47/G47.9_with_date.csv